<img src = "https://chaitimedatascience.com/content/images/2020/07/ctds-1.png">

In [ ]:
import plotly.express as px
import plotly
!pip install dabl -q
import dabl
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import warnings
warnings.filterwarnings('ignore')
import re
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from wordcloud import WordCloud
from pandas_profiling import ProfileReport
import plotly.io as pio
pio.templates.default = "ggplot2"
plt.rcParams['figure.figsize'] = (20,10)


<img src = "https://media.giphy.com/media/Wn74RUT0vjnoU98Hnt/source.gif">

In [ ]:
episodes = pd.read_csv('../input/chai-time-data-science/Episodes.csv',parse_dates=['recording_date','release_date'])
thumbnails = pd.read_csv('../input/chai-time-data-science/YouTube Thumbnail Types.csv')
athumbnails = pd.read_csv('../input/chai-time-data-science/Anchor Thumbnail Types.csv')
description = pd.read_csv('../input/chai-time-data-science/Description.csv')

In [ ]:
episodes.head()

In [ ]:
episodes.info()

In [ ]:
dabl.plot(episodes, 'youtube_views')

In [ ]:
profile = ProfileReport(episodes, title="Pandas Profiling Report")

In [ ]:
profile.to_notebook_iframe()


In [ ]:
null_counts = episodes.isnull().sum()/len(episodes)
plt.figure(figsize=(16,8))
plt.xticks(np.arange(len(null_counts))+0.5,null_counts.index,rotation='vertical')
plt.ylabel('fraction of rows with missing data')
plt.bar(np.arange(len(null_counts)),null_counts)

In [ ]:
episodes.head()

In [ ]:
val_count_gen = episodes['heroes_gender'].value_counts().reset_index()
val_count_gen.columns = ['Gender','count']


In [ ]:
fig = px.bar(val_count_gen, x='Gender', y='count', color='Gender', title='Count of Heroes (Gender Wise)')
fig.show()

In [ ]:
val_count_gen = episodes['heroes_nationality'].value_counts().reset_index()
val_count_gen.columns = ['heroes_nationality','count']
fig = px.bar(val_count_gen, x='heroes_nationality', y='count', color='heroes_nationality', title='Count of Heroes Nationality')
fig.show()

In [ ]:
pd.crosstab(index = episodes['episode_id'],columns=episodes['heroes'], margins=True, margins_name = 'Total').iloc[:10]

In [ ]:
episodes['spotify_streams'] \
    .plot(kind='hist',
          bins=90,
          figsize=(15, 5),
          label='Mens',
          alpha=0.5)
episodes['spotify_listeners'] \
    .plot(kind='hist',
          bins=105,
          figsize=(15, 5),
          label='Womens',
          alpha=0.5)
plt.title('Score Differential')
plt.xlim(0,60)
plt.legend()
plt.show()

In [ ]:
data_to_plot = episodes.dropna(subset=['heroes','youtube_subscribers'])
fig = px.bar(data_to_plot, x='heroes', y='youtube_subscribers', color='heroes_nationality', title='YouTube Subscribers (Heroes)')
fig.show()

In [ ]:
labels = episodes['flavour_of_tea'].value_counts()[:10].index
values = episodes['flavour_of_tea'].value_counts()[:10].values
colors=['#4458bf',
 '#99adbf']

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial',marker=dict(colors=colors))])
fig.update_layout(title="Chai Distribution ",font=dict(size=15,color="#7f7f7f"))
fig.show()

<img src ="https://media.giphy.com/media/yr6mgdF0b9arC/source.gif"> 

In [ ]:
labels = episodes['category'].value_counts()[:10].index
values = episodes['category'].value_counts()[:10].values
colors=['#4458bf',
 '#99adbf']

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial',marker=dict(colors=colors))])
fig.update_layout(title="Category Distribution ",font=dict(size=15,color="#7f7f7f"))
fig.show()

In [ ]:
episodes.head()

In [ ]:
data_to_plot = episodes.dropna()
fig = px.sunburst(data_to_plot, path=['category', 'flavour_of_tea'], values='apple_avg_listen_duration',
                  color='heroes_gender', hover_data=['youtube_subscribers'])
fig.show()

<img src = "https://media.giphy.com/media/3o6Ztm9AHrsloT5LIk/source.gif">

In [ ]:
fig = px.scatter(episodes, x="youtube_views", y="youtube_likes", color="category", marginal_y="violin",
           marginal_x="box", trendline="ols")
fig.show()

In [ ]:
labels = episodes['heroes_kaggle_username'].value_counts()[:10].index
values = episodes['heroes_kaggle_username'].value_counts()[:10].values
fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo='label+percent',
                             insidetextorientation='radial',marker=dict(colors=colors))])
fig.update_layout(title="Kaggle Users Distribution ",font=dict(size=15,color="#7f7f7f"))
fig.show()

In [ ]:
fig = px.scatter(episodes, x="youtube_views", y="youtube_subscribers", color="category", marginal_y="violin",
           marginal_x="box", trendline="ols")
fig.show()

<img src = "https://media.giphy.com/media/KZMRyVjEtdv8AU6mIr/giphy.gif">

In [ ]:
fig = px.scatter_geo(data_to_plot, locations="heroes_location", color="heroes_location", hover_name="heroes_location", size="youtube_views", projection="natural earth")
fig.show()

In [ ]:
sns.countplot(episodes['heroes_nationality'], palette = 'Dark2',hue='category',data=episodes)
plt.title('Nationality wise Category distribution', fontsize = 20)
plt.show()

In [ ]:
labels = ['Morning', 'Evening', 'Afternoon', 'Night']
sizes = episodes['recording_time'].value_counts()
colors = plt.cm.Dark2(np.linspace(0, 1, 4))
explode = [0.1, 0.1, 0.2, 0.5]

plt.rcParams['figure.figsize'] = (9, 9)
plt.pie(sizes, labels = labels, colors = colors, explode = explode, shadow = True)
plt.title('Recoding time for CTDS.show', fontsize = 20)
plt.legend()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(episodes['youtube_views'], color = 'b')
plt.xlabel('Youtube views', fontsize = 16)
plt.ylabel('Number of Views', fontsize = 16)
plt.title('Distribution of Youtube Views', fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

<img src = "https://media.giphy.com/media/aKjaEWISv9Ie4/source.gif"/>

In [ ]:
fig = px.scatter(episodes, x="release_date", y="youtube_views", color="category",
                 size='episode_duration', hover_data=['episode_duration'])
fig.update_layout(title="Release Date vs Youtube Views",font=dict(size=15,color="#7f7f7f"))
fig.show()

In [ ]:
sns.set(style = "dark", palette = "deep", color_codes = True)
x = episodes.episode_duration
plt.figure(figsize = (12,8))
plt.style.use('ggplot')

ax = sns.distplot(x, bins = 10, kde = True, color = 'g')
ax.set_xlabel(xlabel = "Episodes", fontsize = 16)
ax.set_ylabel(ylabel = 'Duration(sec)', fontsize = 16)
ax.set_title(label = 'Histogram Episodes Duration', fontsize = 20)
plt.show()


In [ ]:
sns.set(style = "dark", palette = "deep", color_codes = True)
x = episodes.youtube_avg_watch_duration
plt.figure(figsize = (12,8))
plt.style.use('ggplot')

ax = sns.distplot(x, bins = 10, kde = False, color = 'b')
ax.set_xlabel(xlabel = "youtube_avg_watch_duration", fontsize = 16)
ax.set_ylabel(ylabel = 'Duration(sec)', fontsize = 16)
ax.set_title(label = 'Histogram Youtube watch Duration', fontsize = 20)
plt.show()


In [ ]:
epi_thumb = episodes.merge(thumbnails, left_on="youtube_thumbnail_type", right_on="youtube_thumbnail_type")
epi_thumb["recording_date"] = pd.to_datetime(epi_thumb["recording_date"])
epi_thumb["release_date"] = pd.to_datetime(epi_thumb["release_date"])

In [ ]:
fig = px.scatter(episodes, x="youtube_watch_hours", y="youtube_avg_watch_duration", color="recording_time",
                 size='youtube_watch_hours', hover_data=['episode_duration'])
fig.update_layout(title="YouTube watch vs Youtube Average watch",font=dict(size=15,color="#7f7f7f"))
fig.show()

In [ ]:
fig = px.scatter(episodes, x="youtube_subscribers", y="youtube_likes", color="category",
                 size='youtube_watch_hours', hover_data=['episode_duration'])
fig.update_layout(title="Subscriber vs Youtube likes",font=dict(size=15,color="#7f7f7f"))
fig.show()

In [ ]:
s= episodes.groupby(['heroes','heroes_kaggle_username'])['youtube_watch_hours'].count().reset_index().sort_values(by='youtube_watch_hours',ascending=False)
s.head(10).style.background_gradient(cmap='Oranges')

In [ ]:
s= episodes.groupby(['heroes','heroes_nationality'])['youtube_views'].count().reset_index().sort_values(by='youtube_views',ascending=False)
s.head(10).style.background_gradient(cmap='Blues')

In [ ]:
sns.relplot(x ='episode_duration', y ='youtube_avg_watch_duration', col = 'category', data = episodes, color = 'g', height = 5)

In [ ]:
sns.relplot(x ='youtube_subscribers', y ='youtube_avg_watch_duration', col = 'category', data = episodes, color = 'r', height = 5)

In [ ]:
sns.relplot(x ='youtube_views', y ='youtube_likes', col = 'heroes_gender', data = episodes, color = 'b', height = 5)

<img src = "https://media.giphy.com/media/3o6nV3Ky2C3ZVl0sne/giphy.gif"/>

In [ ]:
sns.relplot(x = 'apple_listeners', y = 'apple_listened_hours', kind = 'line', data=episodes,aspect = 1,height = 7, color = 'r')

In [ ]:
sns.relplot(x = 'apple_listeners', y = 'apple_avg_listen_duration', kind = 'line', data=episodes,aspect = 1,height = 7, color = 'orange')

<img src = "https://media.giphy.com/media/xUA7b2kdiQaSyyAG0o/source.gif"/>

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(episodes.corr(), annot=True, linewidth=0.5, cmap='winter')

In [ ]:
# text preprocessing helper functions

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

In [ ]:
description['description'] = description['description'].apply(str).apply(lambda x: text_preprocessing(x))
wordcloud = WordCloud(
                          background_color='black',
                          max_words=200,
                          max_font_size=40, 
                          random_state=42
                         ).generate(str(description['description']))

plt.figure(figsize=(20, 16))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

<img src = "https://media.giphy.com/media/1lGI0MPO4M0bm/source.gif">

## Cheers!!!☕☕☕☕☕☕